# Data Merger

In this notebook, we will merge the counting statistics, advanced statistics and 
transaction data.

Note, much of the code in this notebook is not optimized and could be 
significantly improved. It would be much better to avoid looping through Pandas 
dataframes row by row, and instead apply more builtin Pandas function, but for 
now it seems to work. 

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from datetime import datetime, timedelta
from nba_api.stats.static import teams

# This ensures side effects do not occur when dealing with views and copies of
# dataframes. See
# https://pandas.pydata.org/pandas-docs/stable//user_guide/copy_on_write.html.
pd.options.mode.copy_on_write = True

pd.options.display.max_columns = None

Load data. 

In [2]:
stats = pd.read_csv('Data/DataToMerge/season_counting_stats.csv')
advanced_stats = pd.read_csv('Data/DataToMerge/advanced_stats.csv')
transaction_data = pd.read_csv('Data/DataToMerge/transaction_data.csv')
nba_key_dates = pd.read_csv('Data/DataToMerge/nba_key_dates.csv')
salary_data = pd.read_csv('Data/DataToMerge/salaries_with_player_id.csv')

Update the columns of the salary_date to change 'Year' to 'SEASON_START'

In [3]:
salary_data = salary_data.rename(columns = {'Year' : 'SEASON_START', 'Team' : 'TEAM_ABBREVIATION'})

Remove data from stats and advanced stats prior to 1990 season because our transaction data cuts off at 1990. 

In [4]:
stats = stats[stats['SEASON_START']>1989]
advanced_stats = advanced_stats[advanced_stats['SEASON_START']>1989]
# Below we will keep nba key dates in 1989 because we want to know about 1990
# offseason.
nba_key_dates = nba_key_dates[nba_key_dates['SEASON_START']>1988]

Organize stats columns.

In [5]:
stats = stats.drop(columns=['SEASON_ID', 'LEAGUE_ID', 'TEAM_ID'])

stats_column_names = stats.columns.to_list()

stats_column_names.remove('NAME')
stats_column_names.remove('SEASON_START')
stats_column_names.insert(0, 'NAME')
stats_column_names.insert(2, 'SEASON_START')

stats = stats[stats_column_names]

stats.head(3)

,NAME,PLAYER_ID,SEASON_START,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
7,Byron Scott,2,1990,LAL,30.0,82,82.0,2630.0,501,1051,0.477,71.0,219.0,0.324,118,148,0.797,54.0,192.0,246.0,177,95.0,21.0,85.0,146,1191
8,Byron Scott,2,1991,LAL,31.0,82,82.0,2679.0,460,1005,0.458,54.0,157.0,0.344,244,291,0.838,74.0,236.0,310.0,226,105.0,28.0,119.0,140,1218
9,Byron Scott,2,1992,LAL,32.0,58,53.0,1677.0,296,659,0.449,44.0,135.0,0.326,156,184,0.848,27.0,107.0,134.0,157,55.0,13.0,70.0,98,792


Organize advanced stats columns. 

In [6]:
advanced_stats = advanced_stats.rename(columns={'tm': 'TEAM_ABBREVIATION'})
advanced_stats = advanced_stats.drop(columns=['player', 'age', 'lg', 'g', 'mp'])
advanced_stats_col_rename_dict = {x: x.upper() for x in advanced_stats.columns}
advanced_stats = advanced_stats.rename(columns=advanced_stats_col_rename_dict)

advanced_stats.head(3)

,PLAYER_ID,SEASON_START,POS,EXPERIENCE,TEAM_ABBREVIATION,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP
0,1631260,2023,SG,2,MIL,10.5,0.617,0.862,0.097,1.6,9.6,5.7,6.4,0.7,0.6,5.5,15.2,0.7,0.3,1.1,0.084,-0.4,-1.7,-2.1,0.0
1,1630639,2023,SG,2,DAL,11.2,0.519,0.413,0.190,4.9,12.6,8.8,8.9,1.5,0.8,9.6,20.0,0.0,0.3,0.2,0.036,-3.1,-1.5,-4.6,-0.2
2,1631100,2023,SF,2,ATL,1.2,0.382,0.629,0.032,1.2,10.6,5.7,3.7,0.3,1.1,11.3,17.3,-0.4,0.0,-0.4,-0.120,-6.0,-3.5,-9.6,-0.3


Fixing mismatched team abbreviations. 

In [7]:
nbs_api_teams = [team["abbreviation"] for team in teams.get_teams()]

print('Teams in stats not found in nba_api:')
for team_abv in stats["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in advanced_stats not found in nba_api:')
for team_abv in advanced_stats["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in transaction_data not found in nba_api:')
for team_abv in transaction_data["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in salary_data not found in nba_api:')
for team_abv in salary_data["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

team_abv_change_dict = {'BRK': 'BKN',
                        'CHH': 'CHA',
                        'CHO': 'CHA',
                        'GOS': 'GSW',
                        'NJN': 'BKN',
                        'NOH': 'NOP',
                        'NOK': 'NOP',
                        'PHL': 'PHI',
                        'PHO': 'PHX',
                        'SAN': 'SAS',
                        'SEA': 'OKC',
                        'UTH': 'UTA',
                        'VAN': 'MEM',
                        'WSB': 'WAS'}

stats["TEAM_ABBREVIATION"] = stats["TEAM_ABBREVIATION"].replace(team_abv_change_dict)
advanced_stats["TEAM_ABBREVIATION"] = advanced_stats["TEAM_ABBREVIATION"].replace(team_abv_change_dict)
stats["TEAM_ABBREVIATION"] = stats["TEAM_ABBREVIATION"].replace(team_abv_change_dict)
salary_data["TEAM_ABBREVIATION"] = salary_data["TEAM_ABBREVIATION"].replace(team_abv_change_dict)

# recheck team abbreviations

print('Teams in stats not found in nba_api:')
for team_abv in stats["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in advanced_stats not found in nba_api:')
for team_abv in advanced_stats["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in transaction_data not found in nba_api:')
for team_abv in transaction_data["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

print('\nTeams in salary_data not found in nba_api:')
for team_abv in salary_data["TEAM_ABBREVIATION"].unique():
    if team_abv not in nbs_api_teams:
        print(team_abv)

Teams in stats not found in nba_api:
VAN
TOT
SEA
SAN
GOS
NJN
PHL
CHH
NOH
UTH
NOK

Teams in advanced_stats not found in nba_api:
TOT
CHO
BRK
PHO
NOH
NJN
SEA
NOK
CHH
VAN
WSB

Teams in transaction_data not found in nba_api:

Teams in salary_data not found in nba_api:
BRK
CHO
PHO
Teams in stats not found in nba_api:
TOT

Teams in advanced_stats not found in nba_api:
TOT

Teams in transaction_data not found in nba_api:

Teams in salary_data not found in nba_api:


Let's do some final checks to see if the stats and advanced stats match up
before merging. 

In [8]:
print(stats.shape)
print(advanced_stats.shape)

(19614, 26)
(19611, 25)


Assuming every (PLAYER_ID, SEASON_START, TEAM_ABBREVIATION) tuple of advanced 
stats is in stats, it appears we are missing 3 rows from advanced stats. 
Let's find them. 

Sarunas Marciulionis (36, 1996, CLE) is missing from advanced stats. It is
strange because every reference I have found (except for NBA.com) does not list 
him as ever having played for Cleveland. It lists him as playing 1 game for 
Cleveland and 16 games for Denver in the 1996 season, but most other references
have him playing for 17 games with Denver. So in stats we will drop the CLE and 
DEN rows and rename the TOT row to DEN (see a few cells below).


In [9]:
stats[stats['PLAYER_ID']==36]

,NAME,PLAYER_ID,SEASON_START,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
206,Sarunas Marciulionis,36,1990,GSW,27.0,50,10.0,987.0,183,365,0.501,1.0,6.0,0.167,178,246,0.724,51.0,67.0,118.0,85,62.0,4.0,75.0,136,545
207,Sarunas Marciulionis,36,1991,GSW,28.0,72,5.0,2117.0,491,912,0.538,3.0,10.0,0.300,376,477,0.788,68.0,140.0,208.0,243,116.0,10.0,193.0,237,1361
208,Sarunas Marciulionis,36,1992,GSW,29.0,30,8.0,836.0,178,328,0.543,3.0,15.0,0.200,162,213,0.761,40.0,57.0,97.0,105,51.0,2.0,76.0,92,521
209,Sarunas Marciulionis,36,1994,OKC,31.0,66,4.0,1194.0,216,457,0.473,35.0,87.0,0.402,145,198,0.732,17.0,51.0,68.0,110,72.0,3.0,98.0,126,612
210,Sarunas Marciulionis,36,1995,SAC,32.0,53,0.0,1039.0,176,389,0.452,64.0,157.0,0.408,155,200,0.775,20.0,57.0,77.0,118,52.0,4.0,96.0,112,571
211,Sarunas Marciulionis,36,1996,CLE,33.0,1,0.0,30.0,3,9,0.333,1.0,3.0,0.333,0,0,0.000,2.0,2.0,4.0,2,0.0,0.0,2.0,3,7
212,Sarunas Marciulionis,36,1996,DEN,33.0,16,0.0,238.0,35,92,0.380,10.0,27.0,0.370,29,36,0.806,10.0,16.0,26.0,23,12.0,1.0,38.0,35,109
213,Sarunas Marciulionis,36,1996,TOT,33.0,17,0.0,268.0,38,101,0.376,11.0,30.0,0.367,29,36,0.806,12.0,18.0,30.0,25,12.0,1.0,40.0,38,116


In [10]:
advanced_stats[advanced_stats['PLAYER_ID']==36]

,PLAYER_ID,SEASON_START,POS,EXPERIENCE,TEAM_ABBREVIATION,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP
16783,36,1996,SG,7,DEN,8.1,0.496,0.297,0.356,5.4,8.0,6.7,16.7,2.4,0.3,25.5,27.5,-0.6,0.1,-0.5,-0.085,-4.0,-0.8,-4.8,-0.2
17328,36,1995,SG,6,SAC,16.7,0.599,0.404,0.514,2.2,6.4,4.3,19.5,2.6,0.3,16.8,24.2,1.9,0.7,2.6,0.120,1.6,-0.3,1.3,0.9
17780,36,1994,SG,5,OKC,14.2,0.562,0.190,0.433,1.7,5.0,3.4,14.1,3.0,0.2,15.3,23.3,1.1,1.2,2.3,0.092,-1.2,0.0,-1.2,0.3
18711,36,1992,SF,4,GSW,20.4,0.618,0.046,0.649,5.3,7.6,6.5,19.0,2.9,0.1,15.3,24.3,2.3,0.6,3.0,0.170,2.4,0.4,2.8,1.0
19169,36,1991,SG,3,GSW,18.8,0.607,0.011,0.523,3.6,7.1,5.4,16.0,2.5,0.3,14.7,24.9,4.9,1.4,6.4,0.144,1.5,-0.2,1.3,1.7
19610,36,1990,SG,2,GSW,16.3,0.576,0.016,0.674,5.7,7.5,6.6,12.0,2.9,0.2,13.7,22.2,1.8,0.7,2.5,0.121,0.0,0.1,0.1,0.5


Chance Comanche	played exactly one game in the 2022-2023 season and did not play
in any other seasons. There is no row in advanced stats for this player. I think 
it is fine to drop this player from the stats data. 

In [11]:
stats[stats['PLAYER_ID']==1628435]

,NAME,PLAYER_ID,SEASON_START,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
27943,Chance Comanche,1628435,2022,POR,27.0,1,0.0,21.0,3,5,0.6,0.0,0.0,0.0,1,4,0.25,2.0,1.0,3.0,0,0.0,1.0,0.0,0,7


In [12]:
advanced_stats[advanced_stats['PLAYER_ID']==1628435]

,PLAYER_ID,SEASON_START,POS,EXPERIENCE,TEAM_ABBREVIATION,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP


In [13]:
stats = stats.drop([211, 212, 27943])
stats.at[213, 'TEAM_ABBREVIATION'] = 'DEN'
stats.at[213, 'TEAM_ABBREVIATION']

'DEN'

Now we check the number of rows of stats and advanced stats and see they are 
equal, which is what we want. 

In [14]:
print(stats.shape)
print(advanced_stats.shape)

(19611, 26)
(19611, 25)


Merge stats and advanced stats. 

In [15]:
merged_data = pd.merge(stats, advanced_stats, on=['PLAYER_ID', 'SEASON_START', 'TEAM_ABBREVIATION'], how='left')

We check the shape and see it has the right number of rows. 

In [16]:
merged_data.shape

(19611, 48)

Organize merged data columns. 

In [17]:
merged_data_columns = merged_data.columns.to_list()

merged_data_columns.remove('EXPERIENCE')
merged_data_columns.insert(5, 'EXPERIENCE')
merged_data_columns.remove('POS')
merged_data_columns.insert(6, 'POS')

merged_data = merged_data[merged_data_columns]

merged_data.head(3)

,NAME,PLAYER_ID,SEASON_START,TEAM_ABBREVIATION,PLAYER_AGE,EXPERIENCE,POS,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP
0,Byron Scott,2,1990,LAL,30.0,8,SG,82,82.0,2630.0,501,1051,0.477,71.0,219.0,0.324,118,148,0.797,54.0,192.0,246.0,177,95.0,21.0,85.0,146,1191,14.2,0.534,0.208,0.141,2.5,8.1,5.4,10.3,1.8,0.5,7.1,19.9,3.6,2.9,6.6,0.120,0.3,0.6,0.9,2.0
1,Byron Scott,2,1991,LAL,31.0,9,SG,82,82.0,2679.0,460,1005,0.458,54.0,157.0,0.344,244,291,0.838,74.0,236.0,310.0,226,105.0,28.0,119.0,140,1218,15.5,0.537,0.156,0.290,3.1,10.2,6.6,13.4,2.0,0.6,9.5,20.4,3.9,2.1,6.0,0.107,1.0,0.1,1.0,2.0
2,Byron Scott,2,1992,LAL,32.0,10,SG,58,53.0,1677.0,296,659,0.449,44.0,135.0,0.326,156,184,0.848,27.0,107.0,134.0,157,55.0,13.0,70.0,98,792,14.4,0.535,0.205,0.279,1.8,7.3,4.6,14.2,1.6,0.5,8.6,20.7,2.4,0.9,3.3,0.094,0.3,-1.0,-0.7,0.5


Next, we collapse the rows from (PLAYER, SEASON, TEAM) to (PLAYER, SEASON). 
We will preserve the teams by creating a list of all teams a player has played 
for during each season.

One could not do this collapse and keep more data, but 
to simplify some of the analysis I will do this collapse. 

In [18]:
# group by PLAYER_ID and SEASON_START
# if only multiple teams in group, take the 'TOT' row
# otherwise, take the single row

merged_data_collapsed_teams = (
    merged_data
    .groupby(['PLAYER_ID', 'SEASON_START'], as_index=False)[merged_data.columns]
    .apply(lambda group: group[group['TEAM_ABBREVIATION']=='TOT']
           if 'TOT' in group['TEAM_ABBREVIATION'].values
           else group)
    .reset_index(drop=True)
)

# edit TEAM_ABBREVIATION to be list of teams, rename to TEAMS_LIST

merged_data_collapsed_teams['TEAM_ABBREVIATION'] = (
    merged_data
    .groupby(['PLAYER_ID', 'SEASON_START'])['TEAM_ABBREVIATION']
    .agg(list)
    .reset_index(drop=True)
)

merged_data_collapsed_teams = merged_data_collapsed_teams.rename(columns={'TEAM_ABBREVIATION': 'TEAMS_LIST'})

# remove 'TOT' from any list of teams

merged_data_collapsed_teams['TEAMS_LIST'] = merged_data_collapsed_teams['TEAMS_LIST'].apply(
    lambda x: x[:-1] if x[-1] == 'TOT' else x
)

merged_data_collapsed_teams.head(3)

,NAME,PLAYER_ID,SEASON_START,TEAMS_LIST,PLAYER_AGE,EXPERIENCE,POS,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP
0,Byron Scott,2,1990,[LAL],30.0,8,SG,82,82.0,2630.0,501,1051,0.477,71.0,219.0,0.324,118,148,0.797,54.0,192.0,246.0,177,95.0,21.0,85.0,146,1191,14.2,0.534,0.208,0.141,2.5,8.1,5.4,10.3,1.8,0.5,7.1,19.9,3.6,2.9,6.6,0.120,0.3,0.6,0.9,2.0
1,Byron Scott,2,1991,[LAL],31.0,9,SG,82,82.0,2679.0,460,1005,0.458,54.0,157.0,0.344,244,291,0.838,74.0,236.0,310.0,226,105.0,28.0,119.0,140,1218,15.5,0.537,0.156,0.290,3.1,10.2,6.6,13.4,2.0,0.6,9.5,20.4,3.9,2.1,6.0,0.107,1.0,0.1,1.0,2.0
2,Byron Scott,2,1992,[LAL],32.0,10,SG,58,53.0,1677.0,296,659,0.449,44.0,135.0,0.326,156,184,0.848,27.0,107.0,134.0,157,55.0,13.0,70.0,98,792,14.4,0.535,0.205,0.279,1.8,7.3,4.6,14.2,1.6,0.5,8.6,20.7,2.4,0.9,3.3,0.094,0.3,-1.0,-0.7,0.5


Now we merge with the salary data

In [19]:
salary_data = salary_data.drop(columns=['Player', 'TEAM_ABBREVIATION', 'Percent_team_salary'])
salary_data = salary_data.rename(columns={'Salary': 'SALARY'})

salary_data.head(3)

,SEASON_START,SALARY,PLAYER_ID
0,1990,3785000.0,73
1,1990,2640000.0,198
2,1990,1400000.0,899


First we aggregate the salaries for a given player and season by summing them.

In [20]:
salary_data = (
    salary_data
    .groupby(['PLAYER_ID', 'SEASON_START'], as_index=False)['SALARY']
    .agg('sum')
    .reset_index(drop=True)
)

salary_data.head(3)

,PLAYER_ID,SEASON_START,SALARY
0,0,1991,250000.0
1,0,1992,36000.0
2,0,1994,316404.0


Now we merge `salary_data` into `merged_data_collapsed_teams` using a left join.

In [21]:
merged_data_collapsed_teams = pd.merge(merged_data_collapsed_teams, salary_data, how='left', on=['PLAYER_ID', 'SEASON_START'])

Let's check how many rows in `merged_data_collapsed_teams` have some missing salary data, and also how many rows have no salary data at all.

In [22]:
print('Percentage of merged_data_collapsed_teams with missing SALARY:',
      len(merged_data_collapsed_teams[merged_data_collapsed_teams['SALARY'].isnull()])/len(merged_data_collapsed_teams))

Percentage of merged_data_collapsed_teams with missing SALARY: 0.04001517162905367


Next, we add the transaction data.

We will also create a new column IN_LEAGUE_NEXT which yields a 1 if we have an entry for that player in our stats data for the following season, and 0 otherwise.

In [23]:
transaction_data.columns

Index(['SEASON_START', 'DATE', 'PLAYER_NAME', 'BBREF_ID', 'TEAM_ABBREVIATION',
       'WAIVED', 'RELEASED', 'TRADED', 'PLAYER_ID'],
      dtype='object')

In [24]:
transaction_data = transaction_data[['PLAYER_ID', 'SEASON_START', 'DATE', 'WAIVED', 'RELEASED', 'TRADED']]

First let's adjust the SEASON_START column of `transaction_data` to reflect the actual NBA season (in other words, if a transaction with a particular SEASON_START occurs after the end of the postseason, we add one to the SEASON_START since we actually want to consider it as occuring in the offseason of the *following* season).

For example, notice how rows 83 and 84 are for transactions occuring during the offseason of 1991, so we want to change SEASON_START for those rows to 1991.

In [25]:
transaction_data.iloc[82:86]

,PLAYER_ID,SEASON_START,DATE,WAIVED,RELEASED,TRADED
82,88,1990,"May 31, 1991",1,0,0
83,76094,1990,"June 21, 1991",1,0,0
84,766,1990,"June 28, 1991",1,0,0
85,705,1991,"July 15, 1991",1,0,0


In [26]:
# define a function which will return the updated SEASON_START for a given row

def update_season_start(row):
    date = datetime.strptime(row["DATE"], '%B %d, %Y').date()
    season_start = row["SEASON_START"]
    post_end = datetime.strptime(
        nba_key_dates.loc[nba_key_dates['SEASON_START'] == season_start, 'POST_END'].iloc[0],
        '%Y-%m-%d'
    ).date()
    if date > post_end:
        return season_start + 1
    return season_start

# apply the above function to the rows of transaction_data

transaction_data["SEASON_START"] = transaction_data.apply(update_season_start, axis=1)

Notice that SEASON_START for rows 83 and 84 have been updated correctly.

In [27]:
transaction_data.iloc[82:86]

,PLAYER_ID,SEASON_START,DATE,WAIVED,RELEASED,TRADED
82,88,1990,"May 31, 1991",1,0,0
83,76094,1991,"June 21, 1991",1,0,0
84,766,1991,"June 28, 1991",1,0,0
85,705,1991,"July 15, 1991",1,0,0


We will add the following column to `transaction_data`.

In [28]:
transaction_data['WAIVED_OFF'] = 0
transaction_data['WAIVED_REG'] = 0
transaction_data['WAIVED_POST'] = 0

transaction_data['RELEASED_OFF'] = 0
transaction_data['RELEASED_REG'] = 0
transaction_data['RELEASED_POST'] = 0

transaction_data['TRADED_OFF'] = 0
transaction_data['TRADED_REG'] = 0
transaction_data['TRADED_POST'] = 0

# define a function which returns the adds a 1 to the appropriate column for a given row

def set_transaction_type(row):
    if row['WAIVED'] == 1:
        transaction_type = 'WAIVED'
    elif row['RELEASED'] == 1:
        transaction_type = 'RELEASED'
    else:
        transaction_type = 'TRADED'

    date = datetime.strptime(row['DATE'], '%B %d, %Y').date()
    season_start = row['SEASON_START']

    reg_start = datetime.strptime(
        nba_key_dates.loc[nba_key_dates['SEASON_START']==season_start, 'REG_START'].iloc[0],
        '%Y-%m-%d').date()
    post_start = datetime.strptime(
        nba_key_dates.loc[nba_key_dates['SEASON_START'] == season_start, 'REG_END'].iloc[0],
        '%Y-%m-%d').date() + timedelta(days=1)

    if date < reg_start:
        row[f'{transaction_type}_OFF'] = 1
    elif date < post_start:
        row[f'{transaction_type}_REG'] = 1
    else:
        row[f'{transaction_type}_POST'] = 1

    return row

# apply the above function to the rows of transaction_data
transaction_data = transaction_data.apply(set_transaction_type, axis=1)

transaction_data.iloc[79:86]

,PLAYER_ID,SEASON_START,DATE,WAIVED,RELEASED,TRADED,WAIVED_OFF,WAIVED_REG,WAIVED_POST,RELEASED_OFF,RELEASED_REG,RELEASED_POST,TRADED_OFF,TRADED_REG,TRADED_POST
79,1027,1990,"April 1, 1991",1,0,0,0,1,0,0,0,0,0,0,0
80,1381,1990,"April 5, 1991",1,0,0,0,1,0,0,0,0,0,0,0
81,76881,1990,"May 21, 1991",1,0,0,0,0,1,0,0,0,0,0,0
82,88,1990,"May 31, 1991",1,0,0,0,0,1,0,0,0,0,0,0
83,76094,1991,"June 21, 1991",1,0,0,1,0,0,0,0,0,0,0,0
84,766,1991,"June 28, 1991",1,0,0,1,0,0,0,0,0,0,0,0
85,705,1991,"July 15, 1991",1,0,0,1,0,0,0,0,0,0,0,0


Before merging, we aggregate the transaction data for player/season pairs.

In [29]:
transaction_data = transaction_data.drop(columns=['DATE'])

transaction_columns = ['WAIVED', 'RELEASED', 'TRADED', 'WAIVED_OFF', 'WAIVED_REG', 'WAIVED_POST',
                       'RELEASED_OFF', 'RELEASED_REG', 'RELEASED_POST', 'TRADED_OFF', 'TRADED_REG', 'TRADED_POST']

transaction_data = (
    transaction_data
    .groupby(['PLAYER_ID', 'SEASON_START'], as_index=False)[transaction_columns]
    .agg('max')
    .reset_index(drop=True)
)

transaction_data.head(3)

,PLAYER_ID,SEASON_START,WAIVED,RELEASED,TRADED,WAIVED_OFF,WAIVED_REG,WAIVED_POST,RELEASED_OFF,RELEASED_REG,RELEASED_POST,TRADED_OFF,TRADED_REG,TRADED_POST
0,2,1993,0,1,0,0,0,0,1,0,0,0,0,0
1,2,1996,1,0,0,1,0,0,0,0,0,0,0,0
2,3,1994,0,0,1,0,0,0,0,0,0,0,1,0


Now we merge `transaction_data` into `merged_data_collapsed_teams` using a left join.

In [30]:
merged_data_collapsed_teams = pd.merge(merged_data_collapsed_teams,
                                       transaction_data,
                                       how='left',
                                       on=['PLAYER_ID', 'SEASON_START'])

# fill null values in transaction columns with 0

merged_data_collapsed_teams[transaction_columns] = merged_data_collapsed_teams[transaction_columns].fillna(0)

merged_data_collapsed_teams.head(3)

,NAME,PLAYER_ID,SEASON_START,TEAMS_LIST,PLAYER_AGE,EXPERIENCE,POS,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED,RELEASED,TRADED,WAIVED_OFF,WAIVED_REG,WAIVED_POST,RELEASED_OFF,RELEASED_REG,RELEASED_POST,TRADED_OFF,TRADED_REG,TRADED_POST
0,Byron Scott,2,1990,[LAL],30.0,8,SG,82,82.0,2630.0,501,1051,0.477,71.0,219.0,0.324,118,148,0.797,54.0,192.0,246.0,177,95.0,21.0,85.0,146,1191,14.2,0.534,0.208,0.141,2.5,8.1,5.4,10.3,1.8,0.5,7.1,19.9,3.6,2.9,6.6,0.120,0.3,0.6,0.9,2.0,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Byron Scott,2,1991,[LAL],31.0,9,SG,82,82.0,2679.0,460,1005,0.458,54.0,157.0,0.344,244,291,0.838,74.0,236.0,310.0,226,105.0,28.0,119.0,140,1218,15.5,0.537,0.156,0.290,3.1,10.2,6.6,13.4,2.0,0.6,9.5,20.4,3.9,2.1,6.0,0.107,1.0,0.1,1.0,2.0,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Byron Scott,2,1992,[LAL],32.0,10,SG,58,53.0,1677.0,296,659,0.449,44.0,135.0,0.326,156,184,0.848,27.0,107.0,134.0,157,55.0,13.0,70.0,98,792,14.4,0.535,0.205,0.279,1.8,7.3,4.6,14.2,1.6,0.5,8.6,20.7,2.4,0.9,3.3,0.094,0.3,-1.0,-0.7,0.5,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Lastly, we add the IN_LEAGUE_NEXT column. Note that all players in the most recent season will have 0 in this column.

In [31]:
# create a set consisting of all (PLAYER_ID, SEASON_START) pairs in merged_data_collapsed_teams

player_season_set = set(merged_data_collapsed_teams[['PLAYER_ID', 'SEASON_START']].itertuples(index=False, name=None))

# define a function which returns whether a given player for a given season shows up in the dataset for the following season

def in_league_next(row):
    return 1.0 * ((row['PLAYER_ID'], row['SEASON_START'] + 1) in player_season_set)


# apply the above function to the rows of merged_data_collapsed_teams to get IN_LEAGUE_NEXT column

merged_data_collapsed_teams['IN_LEAGUE_NEXT'] = (
    merged_data_collapsed_teams[['PLAYER_ID', 'SEASON_START']]
    .apply(in_league_next, axis=1)
)

Let's check this on a single player.

In [32]:
merged_data_collapsed_teams[merged_data_collapsed_teams['PLAYER_ID']==2]

,NAME,PLAYER_ID,SEASON_START,TEAMS_LIST,PLAYER_AGE,EXPERIENCE,POS,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PER,TS_PERCENT,X3P_AR,F_TR,ORB_PERCENT,DRB_PERCENT,TRB_PERCENT,AST_PERCENT,STL_PERCENT,BLK_PERCENT,TOV_PERCENT,USG_PERCENT,OWS,DWS,WS,WS_48,OBPM,DBPM,BPM,VORP,SALARY,WAIVED,RELEASED,TRADED,WAIVED_OFF,WAIVED_REG,WAIVED_POST,RELEASED_OFF,RELEASED_REG,RELEASED_POST,TRADED_OFF,TRADED_REG,TRADED_POST,IN_LEAGUE_NEXT
0,Byron Scott,2,1990,[LAL],30.0,8,SG,82,82.0,2630.0,501,1051,0.477,71.0,219.0,0.324,118,148,0.797,54.0,192.0,246.0,177,95.0,21.0,85.0,146,1191,14.2,0.534,0.208,0.141,2.5,8.1,5.4,10.3,1.8,0.5,7.1,19.9,3.6,2.9,6.6,0.120,0.3,0.6,0.9,2.0,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Byron Scott,2,1991,[LAL],31.0,9,SG,82,82.0,2679.0,460,1005,0.458,54.0,157.0,0.344,244,291,0.838,74.0,236.0,310.0,226,105.0,28.0,119.0,140,1218,15.5,0.537,0.156,0.290,3.1,10.2,6.6,13.4,2.0,0.6,9.5,20.4,3.9,2.1,6.0,0.107,1.0,0.1,1.0,2.0,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Byron Scott,2,1992,[LAL],32.0,10,SG,58,53.0,1677.0,296,659,0.449,44.0,135.0,0.326,156,184,0.848,27.0,107.0,134.0,157,55.0,13.0,70.0,98,792,14.4,0.535,0.205,0.279,1.8,7.3,4.6,14.2,1.6,0.5,8.6,20.7,2.4,0.9,3.3,0.094,0.3,-1.0,-0.7,0.5,1100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Byron Scott,2,1993,[IND],33.0,11,SG,67,2.0,1197.0,256,548,0.467,27.0,74.0,0.365,157,195,0.805,19.0,91.0,110.0,133,62.0,9.0,103.0,80,696,17.6,0.549,0.135,0.356,1.9,8.5,5.3,18.9,2.7,0.5,14.0,27.0,1.7,1.4,3.1,0.123,1.3,0.2,1.5,1.1,650000.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Byron Scott,2,1994,[IND],34.0,12,PG,80,1.0,1528.0,265,583,0.455,79.0,203.0,0.389,193,227,0.850,18.0,133.0,151.0,108,61.0,13.0,119.0,123,802,15.8,0.587,0.348,0.389,1.5,10.3,6.0,12.2,2.1,0.7,14.8,24.0,2.1,1.9,3.9,0.124,1.1,0.3,1.4,1.3,1550000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,Byron Scott,2,1995,[MEM],35.0,13,SG,80,0.0,1894.0,271,676,0.401,74.0,221.0,0.335,203,243,0.835,40.0,152.0,192.0,123,63.0,22.0,100.0,126,819,13.0,0.523,0.327,0.359,2.4,9.8,5.9,11.7,1.8,0.9,11.3,21.3,1.0,1.3,2.3,0.059,-0.8,-0.6,-1.3,0.3,1650000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,Byron Scott,2,1996,[LAL],36.0,14,SG,79,8.0,1440.0,163,379,0.430,73.0,188.0,0.388,127,151,0.841,21.0,97.0,118.0,99,46.0,16.0,53.0,72,526,13.5,0.590,0.496,0.398,1.7,7.6,4.7,10.7,1.7,0.8,10.6,15.5,2.8,1.3,4.1,0.137,0.6,0.4,1.0,1.1,247500.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Finally, we save our DataFrame as a CSV file.

In [33]:
merged_data_collapsed_teams.to_csv('./merged_data_collapsed_teams.csv', index=False)

### Train/test split

In [76]:
from sklearn.model_selection import train_test_split

In [77]:
train_data, test_data = train_test_split(merged_data_collapsed_teams, test_size=0.2, random_state=261)

In [78]:
train_data.to_csv('./train_data.csv', index=False)

In [79]:
test_data.to_csv('./test_data.csv', index=False)